In [ ]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import json

In [ ]:
client = requests.Session()
HOMEPAGE_URL = 'https://www.linkedin.com/uas/login'
#LOGIN_URL = 'https://www.linkedin.com/feed'
html = client.get(HOMEPAGE_URL).content
soup = BeautifulSoup(html,"lxml")
csrf = soup.find('input', type="hidden")['value']

##specify 'cookie' and 'user-agent'
##change the cookie info as shown in your 'inspect' info. Replace value in 'JSESSIONID' with csrf::: '...JSESSIONID={}...'.format(csrf)
headers = {'cookie': 'XXX; \
JSESSIONID={}; XXX'.format(csrf),
        'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Mobile Safari/537.36'}


url= 'https://www.linkedin.com/search/results/index/?keywords={}&origin=HISTORY'.format('3M Drug Delivery Systems')
response = client.get(url=url, headers=headers)
parse_response = BeautifulSoup(response.content,"lxml")

In [ ]:
x = parse_response.find_all('code', style ="display: none")
for i in range(len(x)):
    if 'Silvia' in x[i].get_text():
        print(i)
        break

In [ ]:
data = parse_response.find_all('code', style ="display: none")[i].string
data=json.loads(data)
data = data['included']

pplist = []
for i in data:
    if i['$type'] == 'com.linkedin.voyager.identity.shared.MiniProfile':
        pplist.append(i)
        
pplist

In [ ]:
##requests

##write in function:

def LinkedIn_scraper_company(company):
    client = requests.Session()
    HOMEPAGE_URL = 'https://www.linkedin.com/uas/login'
    #LOGIN_URL = 'https://www.linkedin.com/feed'
    html = client.get(HOMEPAGE_URL).content
    soup = BeautifulSoup(html,"lxml")
    csrf = soup.find('input', type="hidden")['value']


##change the cookie info as yours; replace'JSESSIONID' value in 'cookie with csrf
    headers = {'cookie':'XXXX\
JSESSIONID={}; XXXX'.format(csrf),
        'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Mobile Safari/537.36'}
#login_information = {
    #'session_key':'#####',
    #'session_password':'####',
    #'loginCsrfParam': csrf,
#}

    url= 'https://www.linkedin.com/search/results/index/?keywords={}&origin=HISTORY'.format(company)
    response = client.get(url=url, headers=headers)
    parse_response = BeautifulSoup(response.content,"lxml")
    x = parse_response.find_all('code', style ="display: none")
    data = parse_response.find_all('code', style ="display: none")[10].string
    data=json.loads(data)
    data = data['included']

    pplist = []
    for i in data:
        if i['$type'] == 'com.linkedin.voyager.identity.shared.MiniProfile'\
        and company.strip()[0] in i['occupation']:
            pplist.append(i)
        
    return(pplist)

In [ ]:
LinkedIn_scraper_company('Walmart CEO')

In [ ]:
l = list()
result = LinkedIn_scraper_company('3M Drug Delivery Systems CEO')
for t in result:
    if 'CEO' or 'President' in t['occupation']:
        name = t['firstName']+' '+t['lastName']
        l.append(name)
print(l)

In [ ]:
company = ['Walmart', '3M Drug Delivery Systems', 'Amazon']

In [ ]:
import time
import datetime
n = len(company)
i = 0
list_name = []
while i < n:
    companyname = company[i]
    try:
        result = LinkedIn_scraper_company(companyname+ ' ' + 'CEO')
        for t in result:
            if 'CEO' or 'President' in t['occupation']:
                name = t['firstName']+' '+t['lastName']
                list_name.append(name)
                i+=1
                break
        else:
            list_name.append('NA')
            i+=1
                
    except:
        print(company[i])
        print("Connection refused by the server..")
        print("Let me sleep for 30 seconds")
        print("ZZzzzz...")
        time.sleep(30)
        print("Was a nice sleep, now let me continue...")
        continue
                
    

print(list_name)

In [ ]:
###Selenium




from selenium import webdriver
import time

from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

query_keyword = ['Eneida Boniche Silcott', 'Ippocratis (Ippo) Vrohidis', 'Ethan Han']
print ('Enter the linkedin email')
email= input()
print ("Enter the LinkedIn password")
password= input()

#Open Chrome web 
#driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/login')

#Login bu username/password
email_box = driver.find_element_by_id('username')
email_box.send_keys(email)
pass_box = driver.find_element_by_id('password')
pass_box.send_keys(password)
submit_button = driver.find_element_by_class_name("login__form_action_container")
submit_button.click()

time.sleep(1)

#Find Name and return
def getName(driver):
        nameXpath = "//h1[contains(@class, 'pv-top-card-section')]"
        time.sleep(10)
        name = driver.find_element_by_xpath(nameXpath).text
        return name

def getLocation(driver):
    nameXpath = "//h3[contains(@class, 'pv-top-card-section')]"
    loc = driver.find_element_by_xpath(nameXpath).text
    return loc

def getEducation(driver):
    nameXpath = "//span[contains(@class, 'pv-top-card-v2-section__school')]"
    edu = driver.find_element_by_xpath(nameXpath).text
    return edu

def getConnections(driver):
    nameXpath = "//span[contains(@class, 'pv-top-card-v2-section__connections')]"
    conn = driver.find_element_by_xpath(nameXpath).text
    return conn

def saveAsCSV(data):
    fileName = "linkedin_result.csv"
    f = open(fileName, "a")
    headers="Name,Location,Education,Connections\n"
    f.write(data + '\n')

#For each profile name in query_keywords, retrive name, education, experience and number of connections
for query in query_keyword:
    try:
        driver.get(
            'https://www.linkedin.com/search/results/index/?keywords=' + query)

        xpath = "(//span[text()='" + query + "'])[1]"
        #print (xpath)
        time.sleep(10)
        driver.find_element_by_xpath(xpath).click()
        data = ''
        try :
            name = getName(driver)
            print (name)
            data += name + ','
        except Exception as ex:
            data += 'NA,'

        try:
            loc = getLocation(driver)
            print(loc)
            data += loc + ','
        except Exception as ex:
            data += 'NA,'

        try:
            edu = getEducation(driver)
            print(edu)
            data += edu + ','
        except Exception as ex:
            data += 'NA,'

        try:
            conn = getConnections(driver)
            print(conn)
            data += conn + ','
        except Exception as ex:
            data += 'NA,'

        print (data)
        saveAsCSV(data)
    except Exception as e:
        print("Exception in retrieving data" + e)